# park_piper

This notebook is meant to work out the logic for a commandline Python tool that can take an ArcGIS Online point hosted feature layer and assigns it to the park that it is in.

## Libraries

In [106]:
from arcgis.gis import GIS
from arcgis import features
from arcgis import geometry
import pandas as pd
import geopandas as gpd

from copy import deepcopy
from getpass import getpass

## Variables

In [6]:
PORTAL = "https://skarplab.maps.arcgis.com"
USERNAME = "skarplab"
PIPER_ITEM_ID = "e26633d5fe834e248c0339d34a0f5de8"
PARK_ITEM_ID = "ac9b0c87805c45668d30ce80f7e3b53d"

## Helper Functions

In [135]:
def agol_to_gdf(fset):
    gdf = gpd.read_file(fset.to_geojson)
    gdf.crs = {'init': f'epsg:{fset.spatial_reference["latestWkid"]}'}
    gdf['geometry'] = gdf['geometry'].to_crs(epsg = 4326)
    return gdf

## Authenticate

In [4]:
gis = GIS(PORTAL, "skarplab", getpass())

 ·········


## Load Data

### Parks

In [136]:
parks_item = gis.content.get(PARK_ITEM_ID).layers[0]
parks_fset = parks_item.query()
parks_gdf = agol_to_gdf(parks_fset)
parks_gdf.head()

,OBJECTID,NAME,PARK_TYPE,FILE_NUMBER,PARCEL_COUNT,DEVELOPED,MAP_ACRES,ADDRESS,ZIP_CODE,ALTERNATE_ADDRESS,LWCF,PARKID,INITIAL_AQUISITION_DATE,Shape__Area,Shape__Length,geometry
0,1,Windemere Beaver Dam,Neighborhood,N-27,2,Developed,14.767369,1500 Nottingham Rd,27607,NA,Yes,34,-6.311520e+11,6.432640e+05,11375.221313,"POLYGON ((-78.67111 35.81288, -78.67115 35.812..."
1,2,Walnut Creek Athletic Complex,Metro,METRO-6,9,Developed,104.843799,1201 Sunnybrook Rd,27610,NA,No,35,-4.733856e+11,4.566978e+06,11257.400109,"POLYGON ((-78.57502 35.76375, -78.57511 35.762..."
2,3,Thornton Road Property,Community,C-17,2,Undeveloped,130.609432,5600 Thornton Rd,27616,6100 Thornton Rd,No,1,9.077184e+11,5.689324e+06,13130.109125,"POLYGON ((-78.53732 35.90188, -78.53734 35.900..."
3,4,Mary Belle Pate,Neighborhood,N-44,1,Developed,2.447495,2640 Sierra Dr,27603,NA,No,2,1.251677e+12,1.066124e+05,1412.464218,"POLYGON ((-78.67968 35.75252, -78.67953 35.752..."
4,5,Eliza Pool,Neighborhood,N-31,2,Developed,6.244034,1600 Fayetteville St,27603,NA,No,3,8.255520e+11,2.719890e+05,2738.523454,"POLYGON ((-78.64268 35.75847, -78.64314 35.758..."


### Points

In [138]:
points_item = gis.content.get(PIPER_ITEM_ID).layers[0]
points_fset = points_item.query()
points_gdf = agol_to_gdf(points_fset)
points_gdf.head()

,OBJECTID,GlobalID,CreationDate,Creator,EditDate,Editor,ParkName,geometry
0,1,c233db9f-f8d8-43da-8608-32ffacc75321,1572631793424,skarplab,1572631793424,skarplab,None,POINT (-78.65603 35.76899)
1,2,8ebccb95-fc93-4453-9dc1-9bf040fd09de,1572631801229,skarplab,1572631801229,skarplab,None,POINT (-78.65003 35.79249)
2,3,2a264f37-e889-4bf2-9b67-cad6f1ce1d85,1572631806948,skarplab,1572631806948,skarplab,None,POINT (-78.66964 35.79252)
3,4,60a19999-d939-43bf-a965-6aaea9eaa838,1572631811105,skarplab,1572631811105,skarplab,None,POINT (-78.61441 35.79698)
4,5,8fa686a4-a531-4a4b-a94e-d51badcef9ad,1572631819017,skarplab,1572631819017,skarplab,None,POINT (-78.66445 35.78124)


## Noodles

### Try to do some intersection stuff

In [156]:
# TODO: There's got to be a more idiomatic way to get at the park name here
points_gdf['ParkName'] = points_gdf.apply(lambda x: list(parks_gdf[parks_gdf['geometry'].contains(x['geometry'])]['NAME'])[0], axis = 1)
points_gdf.head()

,OBJECTID,GlobalID,CreationDate,Creator,EditDate,Editor,ParkName,geometry
0,1,c233db9f-f8d8-43da-8608-32ffacc75321,1572631793424,skarplab,1572631793424,skarplab,Dorothea Dix Park,POINT (-78.65603 35.76899)
1,2,8ebccb95-fc93-4453-9dc1-9bf040fd09de,1572631801229,skarplab,1572631801229,skarplab,Fred Fletcher,POINT (-78.65003 35.79249)
2,3,2a264f37-e889-4bf2-9b67-cad6f1ce1d85,1572631806948,skarplab,1572631806948,skarplab,Isabella Cannon,POINT (-78.66964 35.79252)
3,4,60a19999-d939-43bf-a965-6aaea9eaa838,1572631811105,skarplab,1572631811105,skarplab,Lions,POINT (-78.61441 35.79698)
4,5,8fa686a4-a531-4a4b-a94e-d51badcef9ad,1572631819017,skarplab,1572631819017,skarplab,Pullen,POINT (-78.66445 35.78124)
